In [ ]:
import json
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import re
import pandas as pd
import itertools

In [ ]:
with open("articles.json", "r") as f:
    articles = json.load(f)

In [ ]:
articles

# Seperate Analyse jedes heruntergeladenen Feldes

## number_comments

In [ ]:
number_comments_list = [article["number_comments"] for article in articles]
number_comments_np = np.array(number_comments_list)

### Verteilung

In [ ]:
f = plt.figure(figsize=(20,5))
plt.subplot(1, 2, 1)
plt.hist(number_comments_np)
plt.xlabel('Anzahl Kommentare')
plt.ylabel('Häufigkeit')
plt.title('Kommentar Histogram')
plt.subplot(1, 2, 2)
plt.hist(np.sort(number_comments_np)[:12000])
plt.xlabel('Anzahl Kommentare')
plt.ylabel('Häufigkeit')
plt.title('Kommentar Histogram (Zoomed)')
plt.show(f)

In [ ]:
np.mean(number_comments_np)

In [ ]:
f = plt.figure(figsize=(20,5))
plt.subplot(1, 2, 1)
plt.boxplot(number_comments_np)
plt.title('Boxplot Kommentare')
plt.subplot(1, 2, 2)
plt.boxplot(number_comments_np,showfliers=False)
plt.title('Boxplot Kommentare ohne Outlier')
plt.show(f)

### Meist diskutierte Artikel 🚗

In [ ]:
for idx in reversed(np.argsort(number_comments_np)[-5:]):
    print(f"Anzahl Kommentare {articles[idx]['number_comments']}")
    print(f"Überschrift \"{articles[idx]['heading']}\"")
    print(f"Datum {articles[idx]['date']}")
    print()

### Artikel ohne Kommentare

In [ ]:
np.count_nonzero(number_comments_np==0)

## date

In [ ]:
number_comments_list = [article["number_comments"] for article in articles]
number_comments_np = np.array(number_comments_list)

### Artikel pro Jar

Seit 2003 wurden in der Rubrik Wirtschaft Artikel veröffentlicht, allerdings erst ab 2017 mit erhöhter Häufigkeit (> 2000). Seit 2017 steigt die Anzahl der publizierten Artikel (2021 bei bereits der Hälfte des Jahres ca. so viele Artikel wie 2017).

In [ ]:
articles_year = {}
for article in articles:
    date = article["date"]
    date = datetime.strptime(date, "%m-%d-%Y, %H:%M")
    articles_year[date.year] = articles_year.get(date.year,0) + 1

In [ ]:
plt.figure(figsize=(20,5))
plt.bar(range(len(articles_year)), list(articles_year.values()), align='center')
plt.xticks(range(len(articles_year)), list(articles_year.keys()))
plt.xlabel('Jahr')
plt.ylabel('Häufigkeit')
plt.gca().invert_xaxis()
plt.show()

### Artikel nach Monat (ohne 2021)
Im Juli werden am meisten Artikel veröffenlticht (ca. 200 mehr als der Durchschnitt).<br>
Im Februar werden am wenigsten Artikel veröffenlticht (ca. 170 weniger als der Durchschnitt).

In [ ]:
articles_mo = {}
for article in articles:
    date = article["date"]
    date = datetime.strptime(date, "%m-%d-%Y, %H:%M")
    if date.year != 2021:
        articles_mo[date.strftime("%B")] = articles_mo.get(date.strftime("%B"),0) + 1

In [ ]:
plt.figure(figsize=(20,5))
plt.bar(range(len(articles_mo)), list(articles_mo.values()), align='center')
plt.xticks(range(len(articles_mo)), list(articles_mo.keys()))
plt.xlabel('Monat')
plt.ylabel('Häufigkeit')
plt.gca().invert_xaxis()
plt.show()

In [ ]:
articles_mo["July"] - np.mean(list(articles_mo.values()))

In [ ]:
np.mean(list(articles_mo.values())) - articles_mo["February"]

### Artikel nach Stunde (ohne 2021)

Artikel werden gewöhnlich zwischen 6 und 19 Uhr veröffenlticht. Um 10 Uhr werden am meisten Artikel veröffenlicht.

In [ ]:
articles_hour = {}
for article in articles:
    date = article["date"]
    date = datetime.strptime(date, "%m-%d-%Y, %H:%M")
    if date.year != 2021:
        articles_hour[date.hour] = articles_hour.get(date.hour,0) + 1
articles_hour = {key: value for (key, value) in sorted(articles_hour.items(), key=lambda x: x[0], reverse=True)}

In [ ]:
plt.figure(figsize=(20,5))
plt.bar(range(len(articles_hour)), list(articles_hour.values()), align='center')
plt.xticks(range(len(articles_hour)), list(articles_hour.keys()))
plt.xlabel('Stunde')
plt.ylabel('Häufigkeit')
plt.gca().invert_xaxis()
plt.show()

## branding

Es gibt insgesamt 10 verschiedene Tags (Ein Artikel kann nur ein Tag haben; gibt das jeweilige Heise Magazin an). Nur ca. 17% (2715) aller Artikel besitzen ein Tag. Die Tags "Mac & i" und "Autos" treten mit 1012 am häufigsten auf, während das Tag 'Telepolis' nur einmal auftritt.

In [ ]:
tags = [tag for article in articles for tag in article["brandig"]]

In [ ]:
print(len(set(tags)))
set(tags)

In [ ]:
print(f"Anzahl Artikel ohne Tags: {len(articles) - len(tags)}")
print(f"Anzahl Artikel mit Tags: {len(tags)}")
print(f"Anteil Artikel mit Tags: {len(tags)/(len(articles) + len(tags))}")

In [ ]:
tag_frequencies = {}
for tag in tags:
    tag_frequencies[tag] = tag_frequencies.get(tag,0)+1
tag_frequencies = {key: value for (key, value) in sorted(tag_frequencies.items(), key=lambda x: x[1], reverse=True)}

In [ ]:
tag_frequencies

In [ ]:
plt.figure(figsize=(20,5))
plt.bar(range(len(tag_frequencies)), list(tag_frequencies.values()), align='center')
plt.xticks(range(len(tag_frequencies)), list(tag_frequencies.keys()))
plt.xlabel('Tag')
plt.ylabel('Häufigkeit')
plt.gca().invert_xaxis()
plt.show()

## related topics

Ein Artikel kann mehrere related topics, oder auch gar kein related topic haben. Bei ca. 7,5 % fehlt der Eintrag. Es gibt insgesamt 913 verschiedene related topics, wobei "Elektromobilität" die häufigste ist und in insgesamt 11,5 % der Artikel auftaucht.

In [ ]:
related_topics = [article["related_topics"] for article in articles]

In [ ]:
counter=0
# Wie viele leere Elemente sind in der Liste? 
for topic in related_topics:
    if not topic:
        counter += 1
print('Anzahl der Einträge insgesammt:',len(related_topics) ,'Anzahl der leeren Einträge:',counter)   

In [ ]:
related_topics_freq = {}
for topics in related_topics:
    for topic in topics:
        related_topics_freq[topic] = related_topics_freq.get(topic, 0) + 1

In [ ]:
topics_df = pd.DataFrame(related_topics_freq.items(),columns=['topic_name','frequency'])
topics_df = topics_df.sort_values(by='frequency',ascending=False)
topics_df["rel_frequency_article"] = topics_df["frequency"]/len(articles)
topics_df

In [ ]:
topics_df.head(50).plot(kind='bar',x = "topic_name",y='frequency',figsize=(10,10))

### Veränderung der Topics in verschiedenen Jahren

Man erkennt sehr gut, dass die Themen Elektromobilität, Elektoauto, Apple und Geschäftszahlen (im Jahr 2021 auf Platz 7) in allen Jahren eine sehr große Bedeutung gespielt haben. Zusätzlich gibt es weiteren Themen die jeweils in verschiedenen Jahren eine Rolle gespielt haben:

    - 2017 EU
    - 2018 Abgas-Skandal
    - 2019 Mobilfunk
    - 2020 Coronavirus
    - 2021 Coronavirus, Klimawandel

In [ ]:
year_topic = [[datetime.strptime(article["date"], "%m-%d-%Y, %H:%M").year, topic] for article in articles for topic in article["related_topics"]]
year_topic = pd.DataFrame(year_topic, columns=["year","topic"])

In [ ]:
year_topic_freq_df = year_topic.groupby(['year','topic']).size().groupby('year', group_keys=False).nlargest(5).reset_index(name="frequency")
# Nur die Jahre 2017 bis 2021
year_topic_freq_df = year_topic_freq_df[year_topic_freq_df.year.isin(range(2017,2022))]

In [ ]:
year_topic_freq_pivot = year_topic_freq_df.pivot(index="year",columns="topic", values="frequency")
#Sortieren nach NaN Werten, um besseren Barplot zu erhalten
year_topic_freq_pivot = year_topic_freq_pivot[year_topic_freq_pivot.isnull().sum().sort_values(ascending=True).index]

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
#order = ['Elektromobilität','Elektroauto', 'Apple','Geschäftszahlen','EU','Abgas-Skandal', 'Mobilfunk', 'Coronavirus', 'Klimawandel']
year_topic_freq_pivot.plot.bar(stacked=True, ax=ax)
plt.show()

## authors

In den meisten Fällen stammt ein Artikel von nur einem Autor. Neben der dpa (4665) sind Andreas Wilkens(1556), Stefan Krempl(845) und Ben Schwan(712)	die Autoren, die am meisten Artikel geschrieben haben. Wobei Andreas Wilkens (Elektro-)Autos, Stefan Krempl EU und Datenschutz und Ben Schwan Apple thematisiert.

### clean
Manchmal durch "und" oder "," und nicht als neues Listenelement

In [ ]:
authors_article = [article["authors"] for article in articles]

In [ ]:
for i, authors in enumerate(authors_article):
    tmp_elem = []
    for author_s in authors:
        tmp_elem += re.split(',| und |;|/', author_s)
        
    authors_article[i] = [author.strip() for author in tmp_elem]

In [ ]:
for authors in authors_article:
    for i, author in enumerate(authors):
        if '' == author:
            authors.remove(author)
        if '-Korrespondenten' == author:
            authors.remove(author)
        if '. Felix Frieler' == author:
            authors[i] = 'Felix Frieler'
        if 'Grundhoff' == author:
            authors[i] = 'Stefan Grundhoff'
        if 'Martin Fischer (translation by Fabian A. Scherschel)' == author:
            authors[i] = 'Martin Fischer'
            authors.append("Fabian A. Scherschel")
        if author.startswith("Dr."):
            authors[i] = author.split("Dr.")[1][1:]
        if "Stefan 'Krempl" == author:
             authors[i] = 'Stefan Krempl'
        if 'Wolf von Dewitz (dpa)' == author:
            authors[i] = 'Wolf von Dewitz'
        if 'Veronika Szentpetery' == author or 'Veronika Szentpetery-Kessler' == author:
            authors[i] = 'Veronika Szentpétery-Kessler'
        if 'Übersetzung: Ben Schwan' == author:
            authors[i] = 'Ben Schwan'
        if 'press-inform' == author:
            authors[i] = 'pressinform'
            
        material_dpa = ['Mit Material von dpa', 'mit Material von dpa','mt Material von dpa']
        if author in material_dpa:
             authors[i] = 'mit Material der dpa'
        
        dpa = ['den dpa-Korrespondentinnen', 'dpa-AFX', 'dpa-Korrespondenten']
        if author in dpa:
            authors[i] = 'dpa'          

###  Durchschnittliche Anzahl Autoren pro Artikel

In [ ]:
authors_count_per_article = np.array([len(authors) for authors in authors_article])
np.mean(authors_count_per_article)

### Meiste Autoren pro Artikel

In [ ]:
print(np.max(authors_count_per_article))
print(authors_article[np.argmax(authors_count_per_article)])

### Artikel ohne angegebenen Autor

In [ ]:
np.count_nonzero(authors_count_per_article==0)

### Häufigste Autoren

In [ ]:
authors = [author for authors in authors_article for author in authors]
authors_df = pd.DataFrame(authors,columns=['author_name'])

In [ ]:
authors_df = authors_df.groupby(['author_name']).size()
authors_df = pd.DataFrame(authors_df).reset_index()
authors_df = authors_df.rename(columns={0:"number_articles"})
authors_df = authors_df.sort_values(by='number_articles',ascending=False)
authors_df.head(10)

In [ ]:
authors_df.head(20).plot(kind='bar',x = "author_name",y='number_articles',figsize=(10,10))
plt.show()

### Themen der häufigsten Autoren

In [ ]:
author_topics = []
for authors , article in zip(authors_article,articles):
    if "Andreas Wilkens" in authors:
        author_topics += article["related_topics"]
        
pd.DataFrame(author_topics).groupby([0]).size().to_frame().rename(columns={0:"topic_frequency"}).sort_values(by="topic_frequency",ascending=False).head(5)

In [ ]:
author_topics = []
for authors , article in zip(authors_article,articles):
    if "Stefan Krempl" in authors:
        author_topics += article["related_topics"]
        
pd.DataFrame(author_topics).groupby([0]).size().to_frame().rename(columns={0:"topic_frequency"}).sort_values(by="topic_frequency",ascending=False).head(5)

In [ ]:
author_topics = []
for authors , article in zip(authors_article,articles):
    if "Ben Schwan" in authors:
        author_topics += article["related_topics"]
        
pd.DataFrame(author_topics).groupby([0]).size().to_frame().rename(columns={0:"topic_frequency"}).sort_values(by="topic_frequency",ascending=False).head(5)

### Häufige Autoren-Paare

In [ ]:
author_pairs = []
for authors in authors_article:
    author_pairs += list(itertools.combinations(authors, 2))

In [ ]:
author_pairs_df = pd.DataFrame(np.ones(len(author_pairs)))
author_pairs_df["author_pair"] = author_pairs

In [ ]:
author_pairs_df = author_pairs_df.groupby(["author_pair"]).count()
author_pairs_df = author_pairs_df.reset_index()
author_pairs_df = author_pairs_df.rename(columns={0:"number_author_pair"})
author_pairs_df = author_pairs_df.sort_values(by='number_author_pair',ascending=False)
author_pairs_df.head(10)

## heading

In [ ]:
import advertools as adv
import matplotlib.pyplot as pPlot
from wordcloud import WordCloud, STOPWORDS
import numpy as npy
from PIL import Image

In [ ]:
heading_article = [article["heading"] for article in articles]

In [ ]:
dataset = ' '.join(map(str,heading_article))
dataset

In [ ]:
stopwords = adv.stopwords['german']

STOPWORDS.update(stopwords)
wordcloud = WordCloud(background_color="white",width=1920, height=1080, max_words=50).generate(dataset)

plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Bild besser noch etwas größer

In [ ]:
words = 0
for i, heading in enumerate(heading_article):
    result = len(heading.split())
    words = words + result
    
words_average = words/len(heading_article)
print('Durschnittliche Länge der Headings:',words_average)

In [ ]:
# remove stopwords, and numbers, and anything that's not alphanumeric or underscore
pattern = re.compile(r'\b(' + r'|'.join(stopwords) + r')\b\s*')
text = pattern.sub('', dataset)
text = re.sub(r'[^\w]', ' ', text)
newtext = re.sub(r'[0-9]+', '', text)
print(newtext)

In [ ]:
heading_words_list = newtext.split() 
heading_words_df = DataFrame(heading_words_list,columns=['words'])
df = heading_words_df.groupby(['words']).size().to_frame().rename(columns={0:"number_words"})
sorted_df = df.sort_values(by='number_words',ascending=False)
sorted_df.head(10)

## lead

## text